In [15]:
import pandas as pd

df = pd.read_csv("../data/ocr_data.csv")

In [ ]:
#OCR(tesseract)のaccの確認

In [30]:
for index,row in df.iterrows():
    if row["predict_value"] != "NAN":
        df.loc[index,'predict_value'] = int(row["predict_value"])

a = (df["value"] == df["predict_value"])

ocr_acc = a.value_counts()[True]/len(df)
print(ocr_acc)
#0.474

In [34]:
import numpy as np
from PIL import Image
import cv2
import numpy as np

#データ分割用
from sklearn.model_selection import train_test_split
#XGBoost
import xgboost as xgb

In [36]:
y = df["value"].values
img_list = []

In [37]:
for index,path in enumerate(df["image_path"]):
    img_normalized = cv2.imread(path)/255.0
    img_reshaped = img_normalized.reshape(-1)
    img_list.append(img_reshaped)
X = pd.DataFrame(img_list)

In [43]:
train_x, test_x, train_y, test_y = train_test_split(
    X.values, y, test_size=0.2, shuffle=True
    )

dtrain = xgb.DMatrix(train_x, label=train_y)

param = {'max_depth': 2, 'eta': 0.9, 'objective': 'multi:softmax', 'num_class': 71}

num_round = 10
bst = xgb.train(param, dtrain, num_round)

In [63]:
dtest = xgb.DMatrix(test_x)
pred = bst.predict(dtest)

In [71]:
from sklearn.metrics import accuracy_score

score = accuracy_score(test_y, pred)
print('score:{0:.4f}'.format(score))

score:0.9100


In [62]:
#configとmodelの保存
bst.save_model("../models/xgbClassifier.json")
config = bst.save_config()
with open ('../configs/default.json','w') as f:
    f.write(config)

In [70]:
#モデルをロードして分類
with open ('../configs/default.json') as f:
    conf = f.read()

bst2 = xgb.Booster(model_file="../models/xgbClassifier.json")

pred = bst2.predict(dtest)

In [65]:
bst.predict(dtest)

array([66., 28., 14., 59., 63., 41., 35., 29., 66., 14., 30., 36., 42.,
       63.,  4., 42., 39.,  9., 42., 20., 43., 20.,  4., 36., 47., 44.,
       40., 49., 54., 12., 63., 27., 12., 22., 58., 33., 35.,  4., 32.,
       33., 12., 55., 33., 26., 65., 66., 26., 65., 30., 50., 32., 20.,
       34., 14., 60., 48., 29., 32., 37., 43., 46., 35., 14., 50., 65.,
       35., 35., 41., 18., 63., 24., 42.,  0., 43., 52., 67., 52., 33.,
       56., 32., 15., 32., 20., 32., 33., 34., 52., 46., 60.,  6., 19.,
       19.,  3., 34., 35., 51., 16., 19., 56., 68.], dtype=float32)

In [66]:
test_y

array([66, 28, 14, 59, 63, 41, 35, 29, 66, 14, 30, 36, 45, 63,  4, 45, 39,
        9, 45, 20, 43, 20,  4, 36, 47, 44, 40, 49, 54, 13, 63, 27, 12, 22,
       58, 33, 35,  4, 32, 23, 13, 55, 33, 26, 65, 66, 26, 65, 30, 50, 32,
       20, 34, 14, 60, 48, 29, 32, 37, 43, 46, 35, 14, 50, 65, 35, 35, 41,
       18, 63, 24, 45,  0, 43, 52, 67, 52, 23, 56, 32, 15, 32, 20, 32, 13,
       34, 52, 46, 60,  6, 19, 19,  3, 34, 35, 51, 16, 19, 56, 68])

In [69]:
#一枚だけ入力する場合
X.iloc[100]
y[100]
sample = xgb.DMatrix(X.iloc[100].values.reshape(1,768))
pred = bst.predict(sample)
pred

array([44.], dtype=float32)

In [2]:
import time
def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

def cv2pil(image):
    """OpenCV型→PIL型

    Args:
        image(np.ndarray): OpenCVで読んだ画像データ
    Return:
        new_image(PIL.Image.Image):PIL型の画像データ
    """

    new_image = image.copy()
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)
    new_image = Image.fromarray(new_image)
    return new_image